<a href="https://colab.research.google.com/github/lemmynation/DataScienceEcosystem/blob/main/Churn_Experimentation_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing Files**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
pd.options.display.float_format = '{:.2f}'.format
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import  drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
path = '/content/drive/MyDrive/Data Science Masters/2023-2024 Second Sem/Machine Learning/churn.csv'
df = pd.read_csv(path)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Data Science Masters/2023-2024 Second Sem/Machine Learning/churn.csv'

# **Exploratory Data Analysis (EDA)**

In [ ]:
df.head(10)

NameError: name 'df' is not defined

In [ ]:
df.columns

NameError: name 'df' is not defined

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.duplicated().sum()

In [ ]:
df.isnull().any()

In [ ]:
def histogrammer(column_str, median_text=True, **kwargs):    # **kwargs = any keyword arguments
                                                             # from the sns.histplot() function
    median=round(df[column_str].median(), 1)
    plt.figure(figsize=(5,3))
    ax = sns.histplot(x=df[column_str], **kwargs)            # Plot the histogram
    plt.axvline(median, color='red', linestyle='--')         # Plot the median line
    if median_text==True:                                    # Add median text unless set to False
        ax.text(0.25, 0.85, f'median={median}', color='red',
            ha='left', va='top', transform=ax.transAxes)
    else:
        print('Median:', median)
    plt.title(f'{column_str} histogram');

In [ ]:
plt.figure(figsize=(5,1))
sns.boxplot(x=df['income'], fliersize=1)
plt.title('income box plot');

It highlights the concentration of incomes around **60,000 to 100,000**.


In [ ]:
plt.figure(figsize=(5,3))
sns.histplot(x=df['income'])
median = df['income'].median()

In [ ]:
histogrammer('income')

 Revealing a higher count of people with incomes in the lower range. This disparity provides insights into income inequality.

In [ ]:
plt.figure(figsize=(5,1))
sns.boxplot(x=df['overage'], fliersize=1)
plt.title('overage box plot')

In [ ]:
histogrammer('overage')

In [ ]:
plt.figure(figsize=(5,1))
sns.boxplot(x=df['leftover'])
plt.title('left over box plot')

In [ ]:
histogrammer('leftover')

In [ ]:
plt.figure(figsize=(5,1))
sns.boxplot(x=df['house'])
plt.title('house box plot')

In [ ]:
histogrammer('house')

In [ ]:
plt.figure(figsize=(5,1))
sns.boxplot(x=df['handset_price'])
plt.title('handset_price box plot')

In [ ]:
histogrammer('handset_price')

In [ ]:
plt.figure(figsize=(5,1))
sns.boxplot(x=df['over_15mins_calls_per_month'])
plt.title('Over 15mins calls per month box plot')

In [ ]:
histogrammer('over_15mins_calls_per_month')

The majority of call durations fall within the range of 0 to 5 minutes. This suggests that most calls are relatively short. The histogram exhibits a right-skewed distribution, with fewer calls having longer durations.

In [ ]:
plt.figure(figsize=(5,1))
sns.boxplot(x=df['average_call_duration'])
plt.title('Average Call Duration Box Plot')

In [ ]:
histogrammer('average_call_duration')

It reveals a skewed distribution, with most data points concentrated below an overage of 100.

In [ ]:
fig = plt.figure(figsize=(3,3))
data=df['leave'].value_counts()
plt.pie(data,
        labels=[f'{data.index[0]}: {data.values[0]}',
                f'{data.index[1]}: {data.values[1]}'],
        autopct='%1.1f%%'
        )
plt.title('leave counts');

**HANDLING OUTLIER**

In [ ]:
def outlier_imputer(column_name, percentile):
    # Calculate threshold
    threshold = df[column_name].quantile(percentile)
    # Impute threshold for values > than threshold
    df.loc[df[column_name] > threshold, column_name] = threshold

    print('{:>25} | percentile: {} | threshold: {}'.format(column_name, percentile, threshold))

In [ ]:
for column in ['income', 'overage', 'leftover',
               'house', 'handset_price','over_15mins_calls_per_month','average_call_duration']:
               outlier_imputer(column, 0.95)

In [ ]:
df.describe()

In [ ]:
df.corr()

**Duplicating Dataset**

In [ ]:
df_copy = df.copy(deep = True)

In [ ]:
sns.pairplot(df_copy[['income', 'overage', 'leftover','house','handset_price','over_15mins_calls_per_month','average_call_duration','leave']],
             plot_kws={'alpha':0.4, 'size':5},
             );

# **DATA TRANSFORMATION INCLUDING SMOTE**

**Variable Encoding**

In [ ]:
df_copy = pd.get_dummies(df_copy, drop_first=True)

NameError: name 'df_copy' is not defined

In [ ]:
df_copy.head()

NameError: name 'df_copy' is not defined

**Dropping Columns**

In [ ]:
df_copy = df_copy.drop(columns= 'Unnamed: 0')

In [ ]:
df_copy.head()

**Handling Imbalanced Datasets**

In [ ]:
from imblearn.over_sampling import SMOTE, RandomOverSampler
from collections import Counter
import matplotlib.pyplot as plt


x = df_copy.drop(columns=['leave_STAY'])
y = df_copy.leave_STAY # contains the target variable

# The output will show the class distribution before applying SMOTE.
print('before',Counter(y))

smote = SMOTE()
smotex, smotey = smote.fit_resample(x, y)

print('after', Counter(smotey))

plt.bar(Counter(y).keys(), Counter(y).values(), color='blue', label='Before Oversampling')
plt.bar(Counter(smotey).keys(), Counter(smotey).values(), color='orange', label='After Oversampling')
plt.legend()
plt.xlabel('Class')
plt.ylabel('Number of Instances')
plt.title('Class Distribution Before and After Oversampling')
plt.show()


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import * #ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import *

from sklearn import model_selection
from sklearn.model_selection import cross_val_score

In [ ]:
from sklearn.model_selection import train_test_split



xtrain_, xtest_, ytrain_, ytest_ = train_test_split(smotex, smotey, test_size=0.2)

In [ ]:
def get_score(names, models, _X_train, _y_train, _X_test, _y_test, index):
    res = {' ':[index]} # key and a list containing the provided index as the value.
    for n, model in enumerate(models):
      trained = model.fit(_X_train, _y_train)
      score = trained.score(_X_test, _y_test)
      res[names[n]] = score
    df = pd.DataFrame(res).set_index(' ') # Converts the res dictionary into a DataFrame using pandas (pd.DataFrame(res))
    return df

In [ ]:
def get_validators():
  NN = MLPClassifier(max_iter = 1000)
  LR = LogisticRegression(max_iter = 1000)
  SVM = SVC()
  ABC = AdaBoostClassifier()
  GBC = GradientBoostingClassifier()
  KNN = KNeighborsClassifier()
  GNB = GaussianNB()
  ET = ExtraTreesClassifier()
  RF = RandomForestClassifier()
  DT = DecisionTreeClassifier()
  XGB = XGBClassifier()

  return ([     'Neural Network',
                'Logistic Regression',
                'Support Vector Machine',
                'Ada Boost Classifier',
                'Gradient Boosting Classifier',
                'K-Nearest Neighbors',
               'Gaussian Naive Bayes',
               'Extra Trees',
               'Random Forest',
               'Decision Tree',
               'XGB Classifier'],
          [NN, LR, SVM, ABC, GBC, KNN, GNB, ET, RF, DT, XGB])

In [ ]:
names, models = get_validators()
scores = get_score(names, models, xtrain_, ytrain_, xtest_, ytest_, 'Churn_Accu')

scores

# **Another Experimentation Using Standard Scaler and using Cross Validation**

In [ ]:
from imblearn.over_sampling import SMOTE, RandomOverSampler
from collections import Counter

x = df_copy.drop(columns=['leave_STAY'])
y = df_copy.leave_STAY

# define oversampling strategy
smote = SMOTE()
smotex, smotey = smote.fit_resample(x, y)

print(Counter(smotey))

NameError: name 'df_copy' is not defined

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

for col in x:
  smotex[col] = smotex[col].astype(float)
  smotex[[col]] = scaler.fit_transform(smotex[[col]])

smotex.head()

NameError: name 'x' is not defined

In [ ]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split( smotex, smotey, test_size = 0.2 )

from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import * #ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import *

from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from matplotlib.colors import ListedColormap
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import roc_auc_score

In [ ]:
def cross_validate(names, models, _X_train, _y_train, index, cv=10):
  res = []
  for model in models:
    draft = model_selection.cross_val_score(model, _X_train, _y_train, cv=cv)
    res.append(draft)
  index = [index+' CV_' + str(i) for i in range(cv)]
  accuracy = pd.DataFrame(np.array(res).T,
                          columns=names,
                          index=index,
                          ).round(decimals=3)
  return accuracy

In [ ]:
model_names, models = get_validators()
accuracy = cross_validate(model_names, models, xtrain, ytrain, 'Churn_CV')
accuracy

In [ ]:
data = {
    'Model': ['Neural Network', 'Logistic Regression', 'Support Vector Machine', 'Ada Boost Classifier',
              'Gradient Boosting Classifier', 'K-Nearest Neighbors', 'Gaussian Naive Bayes',
              'Extra Trees', 'Random Forest', 'Decision Tree', 'XGB Classifier'],
    'Churn_CV CV_0': [0.64, 0.63, 0.66, 0.66, 0.70, 0.62, 0.62, 0.67, 0.69, 0.61, 0.69],
    'Churn_CV CV_1': [0.65, 0.65, 0.67, 0.67, 0.68, 0.63, 0.66, 0.68, 0.67, 0.62, 0.66],
    'Churn_CV CV_2': [0.64, 0.63, 0.68, 0.67, 0.71, 0.60, 0.61, 0.67, 0.69, 0.63, 0.67],
    'Churn_CV CV_3': [0.66, 0.65, 0.67, 0.67, 0.71, 0.61, 0.64, 0.67, 0.70, 0.62, 0.68],
    'Churn_CV CV_4': [0.66, 0.65, 0.68, 0.67, 0.71, 0.63, 0.63, 0.68, 0.69, 0.64, 0.69],
    'Churn_CV CV_5': [0.68, 0.64, 0.69, 0.67, 0.72, 0.62, 0.62, 0.70, 0.70, 0.63, 0.70],
    'Churn_CV CV_6': [0.65, 0.63, 0.66, 0.65, 0.70, 0.61, 0.61, 0.67, 0.69, 0.62, 0.67],
    'Churn_CV CV_7': [0.66, 0.64, 0.67, 0.67, 0.70, 0.61, 0.64, 0.69, 0.69, 0.61, 0.69],
    'Churn_CV CV_8': [0.65, 0.63, 0.68, 0.66, 0.72, 0.63, 0.62, 0.69, 0.70, 0.63, 0.70],
    'Churn_CV CV_9': [0.66, 0.65, 0.68, 0.66, 0.69, 0.63, 0.63, 0.69, 0.69, 0.60, 0.69]
}

df = pd.DataFrame(data)

# Melt the dataframe to make it suitable for heatmap
melted_df = df.melt(id_vars='Model', var_name='Fold', value_name='Accuracy')

# Plotting the heatmap
plt.figure(figsize=(12, 8))
heatmap = sns.heatmap(melted_df.pivot("Model", "Fold", "Accuracy"), annot=True, cmap="YlGnBu")
plt.title('Accuracy Heatmap Across Folds for Each Model')
plt.show()

In [ ]:
# Data
models = ['Neural Network', 'Logistic Regression', 'Support Vector Machine', 'Ada Boost Classifier',
          'Gradient Boosting Classifier', 'K-Nearest Neighbors', 'Gaussian Naive Bayes',
          'Extra Trees', 'Random Forest', 'Decision Tree', 'XGB Classifier']

accuracy_data = [
    [0.64, 0.65, 0.64, 0.66, 0.66, 0.68, 0.65, 0.66, 0.65, 0.66],
    [0.63, 0.65, 0.63, 0.65, 0.65, 0.64, 0.63, 0.64, 0.63, 0.65],
    [0.66, 0.67, 0.68, 0.67, 0.68, 0.69, 0.66, 0.67, 0.68, 0.68],
    [0.66, 0.67, 0.67, 0.67, 0.67, 0.67, 0.67, 0.67, 0.66, 0.66],
    [0.70, 0.68, 0.71, 0.71, 0.71, 0.72, 0.70, 0.70, 0.72, 0.69],
    [0.62, 0.63, 0.60, 0.61, 0.63, 0.62, 0.61, 0.63, 0.63, 0.63],
    [0.62, 0.66, 0.61, 0.64, 0.63, 0.62, 0.61, 0.64, 0.63, 0.63],
    [0.67, 0.68, 0.67, 0.67, 0.68, 0.70, 0.67, 0.69, 0.69, 0.69],
    [0.69, 0.67, 0.69, 0.70, 0.69, 0.70, 0.69, 0.69, 0.70, 0.69],
    [0.61, 0.62, 0.63, 0.62, 0.64, 0.63, 0.62, 0.61, 0.63, 0.60],
    [0.69, 0.66, 0.67, 0.68, 0.69, 0.70, 0.67, 0.69, 0.70, 0.69]
]

# Calculate mean accuracy for each model
mean_accuracy = np.mean(accuracy_data, axis=1)

# Identify the model with the highest mean accuracy
best_model_index = np.argmax(mean_accuracy)
best_model = models[best_model_index]

# Plotting
plt.figure(figsize=(12, 6))
bars = plt.bar(models, mean_accuracy, color=['skyblue' if model != best_model else 'orange' for model in models])
plt.title('Mean Accuracy Across Models')
plt.xlabel('Models')
plt.ylabel('Mean Accuracy')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()